In [13]:
-------------------------------- MODULE Util --------------------------------
EXTENDS Sequences, FiniteSets, Naturals, TLC

\* Utilies from Practical TLA+
ReduceSet(op(_, _), set, acc) ==
  LET f[s \in SUBSET set] == \* here's where the magic is
    IF s = {} THEN acc
    ELSE LET x == CHOOSE x \in s: TRUE
         IN op(x, f[s \ {x}])
  IN f[set]
ReduceSeq(op(_, _), seq, acc) == 
  ReduceSet(LAMBDA i, a: op(seq[i], a), DOMAIN seq, acc)
\* Pulls an indice of the sequence for elem.
Index(seq, elem) == CHOOSE i \in 1..Len(seq): seq[i] = elem
\* end from Practical TLA+

Range(T) == { T[x] : x \in DOMAIN T }
    
SeqToSet(s) == {s[i] : i \in DOMAIN s}
Last(seq) == seq[Len(seq)]
IsEmpty(seq) == Len(seq) = 0
\* Remove all occurences of `elem` from `seq`
Remove(seq, elem) == SelectSeq(seq, LAMBDA e: e /= elem)

\* Dual to UNION on intersect
INTERSECTION(setOfSets) == ReduceSet(\intersect, setOfSets, UNION setOfSets)

\* Borrowed from Stephan Merz. TLA+ Case Study: A Resource Allocator. [Intern report] A04-R-101 || merz04a, 2004, 20 p. ffinria-00107809f
(* The set of permutations of a finite set, represented as sequences.  *)
PermSeqs(S) ==
  LET perms[ss \in SUBSET S] ==
       IF ss = {} THEN { << >> }
       ELSE LET ps == [ x \in ss |-> 
                        { Append(sq,x) : sq \in perms[ss \ {x}] } ]
            IN  UNION { ps[x] : x \in ss }
  IN  perms[S]

\* Helper to write "unit test" ASSUMES which print when false
test(lhs, rhs) == lhs /= rhs => Print(<<lhs, " IS NOT ", rhs>>, FALSE)


=============================================================================
\* Modification History
\* Last modified Tue Jun 16 12:04:20 CEST 2020 by tim
\* Created Tue Apr 28 16:43:24 CEST 2020 by tim


In [14]:
--------------------------- MODULE ClientCentric ---------------------------
EXTENDS Naturals, TLC, Sequences, FiniteSets, Util
VARIABLES Keys, Values
\* TODO InitValue could be bottom (_|_)

\* TLA+ specifications of Client Centric Isolation Specification by Crooks et al: https://dl.acm.org/doi/10.1145/3087801.3087802
\* TLA+ specifications by Tim Soethout (tim.soethout@ing.com)

\* A database `State` is represented by keys with corresponding values
State == [Keys -> Values]

\* An `Operation` is a read or write of a key and value
Operation == [op: {"read", "write"}, key: Keys, value: Values]

\* Helpers representing Reads and Writes
r(k,v) == [op |-> "read", key |-> k, value |-> v]
w(k,v) == [op |-> "write", key |-> k, value |-> v]

\* A `Transaction` is a total order of `Operation`s
\* Transaction == [ops: Seq(Operation), start: TimeStamp, commit: TimeStamp]
Transaction == Seq(Operation)
\* For simplicity we store start and commit in a lookup function
TimeStamp == Nat
TransactionTimes == [t \in Transaction |-> [start: TimeStamp, commit: TimeStamp]]

\* "An execution e for a set of transactions
\* T is a totally ordered set defined by the pair (Se,−−t \in T−→),
\* where Se is the set of states generated by applying, 
\* starting from the system’s initial state, a permutation of all the transactions in T ."
ExecutionElem == [parentState: State, transaction: Transaction]
\* resultState is the parentState of the next transaction, but not used in the isolation definitions.
\* ExecutionElem == [parentState: State, transaction: Transaction, resultState: State]
\* We represent an `Execution` as a sequence of `Transaction`s with their corresponding parent state.
\* Note: This execution does therefor not contain the "final state" of the execution, since it is not a parent state of a transaction.
Execution == Seq(ExecutionElem)

\* Seq
executionStates(execution) == [ i \in 1..Len(execution) |-> execution[i].parentState ]
\* Set
executionTransactions(execution) == { ep.transaction : ep \in SeqToSet(execution) }

\* "The parent state is the last state in the execution
\* Definition 1: s -T-> s' ≡ [(k,v) ∈ s ∧ (k,v)􏰍 \notin s'] => k ∈ W_T /\ w(k,v) ∈ Σ_T => (k,v) ∈ s.
\* We refer to s as the parent state of T (denoted as sp,T ); to the
\* transaction that generated s as Ts ; and to the set of keys in which
\* s and s′ differ as ∆(s,s′)"
parentState(execution, transaction) == 
  LET ind == CHOOSE i \in 1..Len(execution): execution[i].transaction = transaction
  IN execution[ind].parentState

\* w(k,v) -to-> r(k,v)
\* check reads and writes, implicit because of "write" check in ReadStates
earlierInTransaction(transaction, op1, op2) == Index(transaction, op1) < Index(transaction, op2)

\* state1 -*-> state2
beforeOrEqualInExecution(execution, state1, state2) == 
  LET states == executionStates(execution)
  IN  Index(states, state1) <= Index(states, state2)

\* Read states: from which states can the operation read it's value
ReadStates(execution, operation, transaction) == 
  LET Se == SeqToSet(executionStates(execution))
      sp == parentState(execution, transaction)
  IN { s \in Se:
        /\ beforeOrEqualInExecution(execution, s, sp) \* s -*-> s_p: restrict the valid read states for the operations in T to be no later than sp
        /\ \/ s[operation.key] = operation.value \* (k,v) \in s
           \/ \E write \in SeqToSet(transaction): 
              /\ write.op = "write" /\ write.key = operation.key /\ write.value = operation.value
              /\ earlierInTransaction(transaction, write, operation) \* w(k,v)-to->r(k,v)
\* "By convention, write operations have read states too: for a write operation in T , they include all states in Se up to and including T ’s parent state."
           \/ operation.op = "write"
     }
        
Preread(execution, transaction) ==
  \A operation \in SeqToSet(transaction): ReadStates(execution, operation, transaction) /= {}

PrereadAll(execution, transactions) == 
  \A transaction \in transactions: Preread(execution, transaction)

\* A state `s` is complete for `T` in `e` if every operation in `T` can read from `s`
Complete(execution, transaction, state) == 
  LET setOfAllReadStatesOfOperation(transactionOperations) ==
        { ReadStates(execution, operation, transaction) : operation \in SeqToSet(transactionOperations) }
     \* also include all states for when the transaction contains no operations
      readStatesForEmptyTransaction == { s \in SeqToSet(executionStates(execution)) : beforeOrEqualInExecution(execution, s, parentState(execution, transaction)) }
  IN state \in INTERSECTION(setOfAllReadStatesOfOperation(transaction) \union { readStatesForEmptyTransaction } )

\* "the write set of T comprises the keys that T updates: WT = {k|w(k, v) ∈ ΣT }.
\* For simplicity of exposition, we assume that a transaction only writes a key once."
WriteSet(transaction) == 
  LET writes == { operation \in SeqToSet(transaction) : operation.op = "write" } 
  IN { operation.key : operation \in writes } 
\* "Denoting the set of keys in which s and s′ differ as ∆(s, s′), we express this as NO-CONF_T (s) ≡ ∆(s, sp) ∩ WT = ∅"
NoConf(execution, transaction, state) == 
  LET Sp == parentState(execution, transaction)
      delta == { key \in DOMAIN Sp : Sp[key] /= state[key] }
  IN delta \intersect WriteSet(transaction) = {}
  
\* `t1` comes before `t2` in wall clock/oracle time
ComesStrictBefore(t1, t2, timestamps) == timestamps[t1].commit < timestamps[t2].start

\* Given system state and single transaction (seq of operations), determines new state
effects(state, transaction) == 
       ReduceSeq(LAMBDA o, newState: IF o.op = "write" THEN [newState EXCEPT ![o.key] = o.value] ELSE newState, transaction, state)

\* Lists all possible permutations of executions given set of transactions
executions(initialState, transactions) == 
  \* All possible permutations
  LET orderings == PermSeqs(transactions)
\*      initialState == [k \in Keys |-> InitValue] \* makes it level-1 therefor pass it in
      accummulator == [ execution |-> <<>>, nextState |-> initialState ]
  IN { LET executionAcc == ReduceSeq(
         \*                                store ExecutionElem in accumulator
                            LAMBDA t, acc: [ execution |-> Append(acc.execution, [parentState |-> acc.nextState, transaction |-> t])
  \*                                         calcultate next state
                                           , nextState |-> effects(acc.nextState,t) 
                            ],
                            ordering, accummulator)
\*              recover ExecutionElems
       IN executionAcc.execution
     : ordering \in orderings }

\* Helper: checks if specific execution satisfies given commit test
executionSatisfiesCT(execution, commitTest(_,_)) ==
  LET transactions == executionTransactions(execution)
  IN \A transaction \in transactions: commitTest(transaction, execution)

\* tests there exists an execution for `transactions`, that satisfies the isolation level given by `commitTest`
\* "Definition 5 Given a set of transactions T and their read states, 
\* a storagesystem satisfies an isolation level I iff ∃e:∀t ∈ T :CTI(t,e)."
satisfyIsolationLevel(initialState, transactions, commitTest(_,_)) ==
  \E execution \in executions(initialState, transactions): \A transaction \in transactions:
    \* PrintT(<<"try execution:",execution>>) =>
    commitTest(transaction, execution)

\* Serializability commit test
CT_SER(transaction, execution) ==
  Complete(execution, transaction, parentState(execution, transaction))
Serializability(initialState, transactions) == satisfyIsolationLevel(initialState, transactions, CT_SER)

\*SerializabilityDebug(initialState, transactions) == 
\*  \* if no executions satify commit test, print all executions
\*  \/ (~\E execution \in executions(initialState, transactions): \A transaction \in transactions:
\*       CT_SER(transaction, execution)) => \A execution \in executions(initialState, transactions): PrintT(<<"Execution not Serializable:",execution>>)
\*  \* fall back to normal check
\*  \/ \E execution \in executions(initialState, transactions): \A transaction \in transactions: CT_SER(transaction, execution)
  
\* Prints the executions. When not Serializable, then there exists no execution which is serializable => All executions are not serializable.
NonSerializableExecutions(initialState, transactions) == 
  ~ Serializability(initialState, transactions) => Print(<<"Executions not Serializable:",  executions(initialState, transactions)>>, FALSE)
\*   When Serializable, print serializable executions
SerializableExecutions(initialState, transactions) == 
  Serializability(initialState, transactions) => Print(<<"Serializable Executions:", 
        { execution \in executions(initialState, transactions) : \A transaction \in transactions: CT_SER(transaction, execution)  } 
    >>, TRUE)
SerializabilityDebug(initialState, transactions) == SerializableExecutions(initialState, transactions) /\ NonSerializableExecutions(initialState, transactions)

\* Snapshot Isolation
CT_SI(transaction, execution) == \E state \in SeqToSet(executionStates(execution)):
  Complete(execution, transaction, state) /\ NoConf(execution, transaction, state)
SnapshotIsolation(initialState, transactions) == satisfyIsolationLevel(initialState, transactions, CT_SI)

\* Strict Serializability: ∀T ∈T:T <s T => s_T′ -*-> s_T.
CT_SSER(timestamps, transaction, execution) ==
  LET Sp == parentState(execution, transaction)
  IN /\ Complete(execution, transaction, Sp)
     /\ \A otherTransaction \in executionTransactions(execution): 
        ComesStrictBefore(otherTransaction, transaction, timestamps) => 
          beforeOrEqualInExecution(execution, parentState(execution, otherTransaction), Sp)
\* For now inline `satisfyIsolationLevel` instead of `satisfyIsolationLevel(transactions, CT_SSER(timestamps)) because partial functions are not supported/hard`
StrictSerializability(initialState, transactions, timestamps) ==
  \E execution \in executions(initialState, transactions): \A transaction \in transactions: CT_SSER(timestamps, transaction, execution)

\* Read Committed
CT_RC(transaction, execution) == Preread(execution, transaction)
ReadCommitted(initialState, transactions) == satisfyIsolationLevel(initialState, transactions, CT_RC)

\* Read Uncommitted
CT_RU(transaction, execution) == TRUE
ReadUncommitted(initialState, transactions) == satisfyIsolationLevel(initialState, transactions, CT_RU)

\* Check types in derived specification
TypeOKT(transactions) ==
\*  /\ InitValue \in Values
  /\ transactions \subseteq Transaction

TypeOK(transactions, execution) == 
  /\ TypeOKT(transactions)
\*  /\ PrintT(State)
  /\ execution \in Execution

=============================================================================

In [15]:
--------------------------- MODULE ClientCentricPaperExamples ---------------------------
EXTENDS ClientCentric, Sequences, FiniteSets, TLC, Integers

x == "x"
y == "y"
z == "z"
keys == {x,y,z}

\* example figure 2
s0 == [k \in keys |-> 0] \* equivalent to ("x" :> 0) @@ ("y" :> 0) @@ ("z" :> 0)
s1 == (x :> 1) @@ (y :> 0) @@ (z :> 0)
s2 == (x :> 1) @@ (y :> 1) @@ (z :> 0)
s3 == (x :> 1) @@ (y :> 2) @@ (z :> 1)

\* operations
r2 == r(y,1) \* [op |-> "read", key |-> "y", value |-> 1]
r3 == r(z,0) \* [op |-> "read", key |-> "z", value |-> 0]
r4 == r(x,0) \* [op |-> "read", key |-> "x", value |-> 0]
r5 == r(z,1) \* [op |-> "read", key |-> "z", value |-> 1]
w1 == w(x,1) \* [op |-> "write", key |-> "x", value |-> 1]

\* transactions
ta == <<w1>>
tb == <<r2, r3>>
tc == <<w(y,1)>>
td == <<
  w(y,2),
  w(z,1)
>>
te == << r4, r5 >>

e1 == [parentState |-> s0, transaction |-> ta ]
e2 == [parentState |-> s2, transaction |-> tb ]
e3 == [parentState |-> s1, transaction |-> tc ]
e4 == [parentState |-> s2, transaction |-> td ]
e5 == [parentState |-> s3, transaction |-> te ]

exampleExecution  == << e1,e3,e2,e4,e5 >>
exampleStates == << s0,s1,s2,s2,s3 >>
transactions == {ta,tb,tc,td,te}

\* TypeOK == /\ transactions \subseteq Transaction
\*           /\ exampleExecution \subseteq Execution

ExampleTypeOK == TypeOK(transactions, exampleExecution) 

\* to let model checker run
Init == FALSE /\ Keys = 0
Next == FALSE /\ Keys' = Keys

ASSUME test(executionStates(exampleExecution), exampleStates)
\* parentState
ASSUME test(parentState(exampleExecution, ta), s0)
ASSUME test(parentState(exampleExecution, tc), s1)
ASSUME test(parentState(exampleExecution, td), s2)

ASSUME test(ReadStates(exampleExecution, w1, ta), { s0 })

ASSUME test(WriteSet(ta), {x})

\* examples from paper
\* "Looking again at Figure 2, s2 is a complete state for transaction Tb , as it is in the set of candidate read states of both r2 (y,y1 ) ({s2 }) and r3(z,z0) ({s0,s1,s2})."
ASSUME test(ReadStates(exampleExecution, r2, tb), { s2 })
ASSUME test(ReadStates(exampleExecution, r3, tb), { s0,s1,s2 })
ASSUME test(Complete(exampleExecution, tb, s2), TRUE)
ASSUME test(SeqToSet(executionStates(exampleExecution)), {s0,s1,s2,s3})
ASSUME \E state \in SeqToSet(executionStates(exampleExecution)):
  Complete(exampleExecution, tb, state)

\* "A complete state is not guaranteed to exist: no such state exists for Te , as the sole candidate read states of r4 and r5 (respectively, s0 and s3) are distinct."
ASSUME test(ReadStates(exampleExecution, r4, te), { s0 })
ASSUME test(ReadStates(exampleExecution, r5, te), { s3 })
ASSUME test(SeqToSet(executionStates(exampleExecution)), {s0,s1,s2,s3})
ASSUME Complete(exampleExecution, te, s0) = FALSE
ASSUME Complete(exampleExecution, te, s1) = FALSE
ASSUME Complete(exampleExecution, te, s2) = FALSE
ASSUME Complete(exampleExecution, te, s3) = FALSE
ASSUME ~\E state \in SeqToSet(executionStates(exampleExecution)): 
  Complete(exampleExecution, te, state)
ASSUME test(executionSatisfiesCT(exampleExecution, CT_SER), FALSE)

\* extra tests
transactionWithoutTe == {ta,tb,tc,td}
executionWithoutTe == << e1,e3,e2,e4 >>
ASSUME test(parentState(executionWithoutTe, ta), s0)
\* "By convention, write operations have read states too: for a write operation in T , they include all states in Se up to and including T's parent state."
ASSUME test(ReadStates(executionWithoutTe, w1, ta), { s0 })
ASSUME Complete(executionWithoutTe, ta, s0) = TRUE
ASSUME test(executionTransactions(executionWithoutTe), transactionWithoutTe)
ASSUME test(executionSatisfiesCT(executionWithoutTe, CT_SER), TRUE)

\* check for which isolation level all transaction in example hold
ASSUME test(executionSatisfiesCT(exampleExecution, CT_SER), FALSE)
ASSUME test(executionSatisfiesCT(exampleExecution, CT_SI), FALSE)
ASSUME test(executionSatisfiesCT(exampleExecution, CT_RC), TRUE)
ASSUME test(executionSatisfiesCT(exampleExecution, CT_RU), TRUE)

\* PermSeqs
ASSUME test(PermSeqs({"t1","t2","t3"}), {<<"t1", "t2", "t3">>, <<"t1", "t3", "t2">>, <<"t2", "t1", "t3">>, <<"t2", "t3", "t1">>, <<"t3", "t1", "t2">>, <<"t3", "t2", "t1">>})
\* should give all possible executions given transactions
execE1 == << [parentState |-> s0, transaction |-> ta ], [parentState |-> s1, transaction |-> tb ] >>
execE2 == << [parentState |-> s0, transaction |-> tb ], [parentState |-> s0, transaction |-> ta ] >>
ASSUME test(executions(s0, {ta,tb}), { execE1,execE2 })

\* Isolation tests for example transactions (exists a execution)
\* Execution found by Serializability(s0, transactions)
exampleSerializableExecutionCP ==               
   << [ parentState |-> [x |-> 0, y |-> 0, z |-> 0],
        transaction |-> <<[op |-> "write", key |-> "y", value |-> 1]>> ],
      [ parentState |-> [x |-> 0, y |-> 1, z |-> 0],
        transaction |->
            << [op |-> "read", key |-> "y", value |-> 1],
               [op |-> "read", key |-> "z", value |-> 0] >> ],
      [ parentState |-> [x |-> 0, y |-> 1, z |-> 0],
        transaction |->
            << [op |-> "write", key |-> "y", value |-> 2],
               [op |-> "write", key |-> "z", value |-> 1] >> ],
      [ parentState |-> [x |-> 0, y |-> 2, z |-> 1],
        transaction |->
            << [op |-> "read", key |-> "x", value |-> 0],
               [op |-> "read", key |-> "z", value |-> 1] >> ],
      [ parentState |-> [x |-> 0, y |-> 2, z |-> 1],
        transaction |-> <<[op |-> "write", key |-> "x", value |-> 1]>> ] >>
exampleSerializableExecution ==               
   << [ parentState |-> s0,
        transaction |-> tc ],
      [ parentState |-> [x |-> 0, y |-> 1, z |-> 0],
        transaction |-> tb ],
      [ parentState |-> [x |-> 0, y |-> 1, z |-> 0],
        transaction |-> td ],
      [ parentState |-> [x |-> 0, y |-> 2, z |-> 1],
        transaction |-> te ],
      [ parentState |-> [x |-> 0, y |-> 2, z |-> 1],
        transaction |-> ta ] >>
ASSUME test(exampleSerializableExecutionCP, exampleSerializableExecution)
ASSUME test(executionSatisfiesCT(exampleSerializableExecution, CT_SER), TRUE)
ASSUME test(Serializability(s0, transactions), TRUE)
ASSUME test(SnapshotIsolation(s0, transactions), TRUE)
ASSUME test(ReadCommitted(s0, transactions), TRUE)
ASSUME test(ReadUncommitted(s0, transactions), TRUE)

\* start and commit time stamps are not specified by the example 
\* ASSUME test(CT_SSER(transactions, exampleExecution), FALSE)

\* Simple Banking Example (figure 3)
\* example A
S == "S"
C == "C"
\* states
as1 == (C :>  30) @@ (S :> 30)
as2 == (C :> -10) @@ (S :> 30)
\* as3 == ("C" :> 30) @@ ("S" :> 30) FAIL

\* transactions
talice == << r(S,30), r(C, 30), w(C,-10) >>
tbob   == << r(S,30), r(C,-10) (* w(S,-10) Write does not happen *) >>

\* execution
ae1 == [parentState |-> as1, transaction |-> talice ]
ae2 == [parentState |-> as2, transaction |-> tbob ]
aExecution  == << ae1, ae2 >>

bankTransactions == {talice, tbob}

BankExampleATypeOK == TypeOK(bankTransactions, aExecution)

ASSUME test(parentState(aExecution, talice), as1)
ASSUME Complete(aExecution, talice, as1)
ASSUME Complete(aExecution, tbob, as2)
\* test isolation levels
ASSUME test(executionSatisfiesCT(aExecution, CT_SER), TRUE) 
ASSUME test(executionSatisfiesCT(aExecution, CT_SI), TRUE)
ASSUME test(executionSatisfiesCT(aExecution, CT_RC), TRUE)
ASSUME test(executionSatisfiesCT(aExecution, CT_RU), TRUE)

ASSUME test(Serializability(as1, bankTransactions), TRUE)
ASSUME test(SnapshotIsolation(as1, bankTransactions), TRUE)
ASSUME test(ReadCommitted(as1, bankTransactions), TRUE)
ASSUME test(ReadUncommitted(as1, bankTransactions), TRUE)

\* talices comes strictly before tbob
bankTimeStamps == (talice :> [start |-> 1, commit |-> 2]) @@ (tbob :> [start |-> 3, commit |-> 4])
ASSUME test(StrictSerializability(as1, bankTransactions, bankTimeStamps), TRUE) 

\* example B
bs1 == (C :>  30) @@ (S :>  30)
bs2 == (C :> -10) @@ (S :>  30)
bs3 == (C :> -10) @@ (S :> -10)

\* transactions
tbAlice == << r(S, 30), r(C,30), w(C,-10) >>
tbBob   == << r(S, 30), r(C,30), w(S,-10) >>
\* tread == <<
\*   [op |-> "read", key |-> "S", value |-> -10],
\*   [op |-> "read", key |-> "C", value |-> -10]
\* >>

bbe1 == [parentState |-> bs1, transaction |-> tbAlice ]
bbe2 == [parentState |-> bs2, transaction |-> tbBob ]
\* bbe3 == [parentState |-> bs3, transaction |-> tread ]
bExecution  == << bbe1, bbe2>>

bBankTransactions == {tbAlice, tbBob}

BankExampleBTypeOK == TypeOK(bBankTransactions, bExecution)

ASSUME test(executionSatisfiesCT(bExecution, CT_SER), FALSE)  \* not Serializable 
ASSUME test(Serializability(as1, bBankTransactions), FALSE)

\* prerequisits for SI
ASSUME \E state \in SeqToSet(executionStates(bExecution)):
  Complete(bExecution, tbAlice, state) \* bs1
ASSUME \E state \in SeqToSet(executionStates(bExecution)):
  Complete(bExecution, tbBob, state) \* bs1
ASSUME NoConf(bExecution, tbAlice, bs1)
ASSUME NoConf(bExecution, tbBob, bs1)
ASSUME test(executionSatisfiesCT(bExecution, CT_SI), TRUE)  \* but is allowed under SI
ASSUME test(SnapshotIsolation(bs1, bBankTransactions), TRUE)

ASSUME test(executionSatisfiesCT(bExecution, CT_RC), TRUE)
ASSUME test(executionSatisfiesCT(bExecution, CT_RU), TRUE)
ASSUME test(ReadCommitted(bs1, bBankTransactions), TRUE)
ASSUME test(ReadUncommitted(bs1, bBankTransactions), TRUE)

\* Some extra tests on CC helper functions 
ASSUME test(effects(s0, <<>>), s0)
ASSUME test(effects(s0, tb), s0)
ASSUME test(effects(s0, ta), s1)
ASSUME test(effects(s1, tc), s2)
ASSUME test(effects(s2, td), s3)

\* test with empty operation
ASSUME test(executions(s0, {<<>>}), {<<[parentState |-> s0, transaction |-> <<>>]>>})

\* Empty transactions should also be Serializable, because they can read from all states
InitalState == ("r1" :> 0 @@ "r2" :> 0)
ccTransactions2 == {<<>>}

emptyTransaction == <<>>
emptyExecution == <<[parentState |-> InitalState, transaction |-> emptyTransaction ]>>

ASSUME Preread(emptyExecution, emptyTransaction) \* TRUE as expected because of \forall operations, so empty operations means TRUE
ASSUME CT_SER(emptyTransaction, emptyExecution)
ASSUME Serializability(InitalState, ccTransactions2)



\* Debug serializability 2PL/2PC
2plr1 == "r1"
2plr2 == "r2"
2plt1 == << r(2plr1,0), r(2plr2,1) >>
2plt2 == << r(2plr1,0), w(2plr1,1), r(2plr2,0), w(2plr2,1) >>
2ple1 == << [ parentState |-> (2plr1 :> 0 @@ 2plr2 :> 0), transaction |-> << r(2plr1,0), r(2plr2,1) >> ],
            [ parentState |-> (2plr1 :> 0 @@ 2plr2 :> 0), transaction |-> << r(2plr1,0), w(2plr1,1), r(2plr2,0), w(2plr2,1) >> ]  >>
2ple2 == << [ parentState |-> (2plr1 :> 0 @@ 2plr2 :> 0), transaction |-> << r(2plr1,0), w(2plr1,1), r(2plr2,0), w(2plr2,1) >> ],
            [ parentState |-> (2plr1 :> 1 @@ 2plr2 :> 1), transaction |-> << r(2plr1,0), r(2plr2,1) >> ] >>     
2plEs == { 2ple1, 2ple2 }

ASSUME CT_SER(2plt1, 2ple1) = FALSE
ASSUME CT_SER(2plt2, 2ple1) = TRUE

ASSUME CT_SER(2plt1, 2ple2) = FALSE
ASSUME CT_SER(2plt2, 2ple2) = TRUE

ASSUME Serializability(InitalState, {2plt1, 2plt2}) = FALSE
\* As expected, because `r(2plr2,1)` makes it non-serializable.

=============================================================================


In [16]:
%tlc:ClientCentricPaperExamples
CONSTANTS
  Keys = {x,y,z}
  Values = {x0,x1, y0,y1,y2, z0,z1}

INIT Init
NEXT Next

INVARIANTS
  ExampleTypeOK
  BankExampleATypeOK
  BankExampleBTypeOK

TLC2 Version 2.14 of 10 July 2019 (rev: 0cae24f)
Running breadth-first search Model-Checking with fp 11 and seed 578679069238585170 with 8 workers on 8 cores with 443MB heap and 64MB offheap memory [pid: 297] (Linux 4.19.76-linuxkit amd64, AdoptOpenJDK 14.0.2 x86_64, MSBDiskFPSet, DiskStateQueue).
Parsing file /tmp/tmpr4vlqwmy/ClientCentricPaperExamples.tla
Parsing file /tmp/tmpr4vlqwmy/ClientCentric.tla
Parsing file /tmp/Sequences.tla
Parsing file /tmp/FiniteSets.tla
Parsing file /tmp/TLC.tla
Parsing file /tmp/Integers.tla
Parsing file /tmp/Naturals.tla
Parsing file /tmp/tmpr4vlqwmy/Util.tla
Semantic processing of module Naturals
Semantic processing of module Sequences
Semantic processing of module FiniteSets
Semantic processing of module TLC
Semantic processing of module Util
Semantic processing of module ClientCentric
Semantic processing of module Integers
Semantic processing of module ClientCentricPaperExamples
Starting... (2020-09-16 14:02:22)
Computing initial states...
Finished 